In [1]:
from typing import List, Set, Tuple
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [2]:
excel_path  = r"C:\Users\issam\Desktop\Files\PhD\DRPRL\data\StaticProblemInstances\GroupA\distancematrix_with_20_nodes(seconds).xlsx"
D1 = pd.read_excel(excel_path, sheet_name='D1', header = None).values
battery_capacity = 1800


c:\Users\issam\Desktop\Files\PhD\DRPRL\venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Model

In [11]:
model = gp.Model("DRP")
model.setParam('TimeLimit', 300)

Set parameter TimeLimit to value 300


Variables

In [12]:
n = len(D1)
x = {}
for i in range(n):
    for j in range(n):  
        if i != j:
            x[i,j] = model.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}')

In [13]:
# Print variable count
print(f"Number of variables: {len(x)}")

Number of variables: 600


objective function 

In [14]:
obj = gp.quicksum(D1[i,j]*x[i,j] for i,j in x)
model.setObjective(obj, GRB.MINIMIZE)

constraints <br> flow conservation

In [15]:
for h in range(20):
    model.addConstr(gp.quicksum(x[i,h] for i in range(n) if i != h) == 
                    gp.quicksum(x[h,j] for j in range(n) if h != j))

visit each customer once 

In [16]:
for h in range(20) :
    model.addConstr(gp.quicksum(x[i,j] for i in range(n) if i!=j) == 1)

Battery constraints


In [17]:
V = set(range(n))
battery_var = model.addVars(V, vtype=GRB.CONTINUOUS, name='battery')

for i,j in x:
    if D1[i,j] > battery_capacity:
        model.addConstr(x[i,j] == 0)

for i,j in x : 
    model.addConstr(battery_var[j] <= battery_var[i] - D1[i,j] * x[i,j])

# Add stations recharging
stations = set(range(20, 25))  # Last 5 nodes are stations
for i,j in x:
    if j in stations:  # If arriving at station
        model.addConstr(battery_var[j] == battery_capacity)  # Full recharge

Solve

In [18]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13620H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 808 rows, 625 columns and 3408 nonzeros
Model fingerprint: 0x5991ca5b
Variable types: 25 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e-02, 3e+03]
  Objective range  [1e-02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 768 rows and 625 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [ ]:
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective: {model.objVal}")
    print("\nSelected arcs:")
    for i,j in x:
        if x[i,j].X > 0.5:
            print(f"{i} -> {j} (cost: {D1[i,j]})")
else: 
    print("No solution found")

In [5]:
!python -m pip install -U matplotlib


[notice] A new release of pip available: 22.3 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import matplotlib.pyplot as plt 
import networkx as nx

In [9]:
for i,j in x :
    if x[i,j].X > 0.5 :
        print(i,j)

NameError: name 'x' is not defined